# Scraper Feature
## Scraping TripAdvisor Reviews for further analysis and processing

This feature takes a TripAdvisor venue's homepage as inputs and scrapes all the reviews related to that venue and stores the data for further processing.

Instructions:
1. Update the `dataset_name_stem` in the Inputs & Setup section below to whatever the dataset should be referred to throughout the tool.  For example, if you are scraping and analysing reviews for the Three Broomsticks venue in Diagon Alley, then name the dataset `3B` and update the `dataset` variable in the `config.toml` file before doing further processing. Make sure these folders are setup with the proper name and an appropriate `line-config.toml` file as well
2. Update the `start_urls` to include the URLs that should be scraped. A single venue, or multiple datasets in a venue are possible.

## Inputs & Setup

In [12]:
import requests
from bs4 import BeautifulSoup
import csv
import webbrowser
import io
import re
import time

# Setup Variables
dataset_name_stem = 'uo_wide'

start_urls = [
    'https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida.html'
]

lang = 'en'


## Trip Advisor Scraping with BS4

- todo: move to separate file and import

In [13]:
def display(content, filename='output.html'):
    with open(filename, 'wb') as f:
        f.write(content)
        webbrowser.open(filename)

In [14]:
def get_soup(session, url, show=False):
    r = session.get(url)
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200:  # not OK
        print('[get_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')

In [15]:
def post_soup(session, url, params, show=False):

    r = session.post(url, data=params)

    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200:  # not OK
        print('[post_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')

In [16]:
def scrape(url, lang='ALL'):
    # create session to keep all cookies (etc.) between requests
    session = requests.Session()

    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:57.0) Gecko/20100101 Firefox/57.0',
    })

    items = parse(session, url + '?filterLang=' + lang)

    return items

In [17]:
def parse(session, url):
    '''Get number of reviews and start getting subpages with reviews'''

    soup = get_soup(session, url)

    if not soup:
        print('[parse] No Soup:', url)
        return

    num_reviews = soup.find('span', class_='reviews_header_count').text  # get text
    num_reviews = num_reviews[1:-1]
    num_reviews = num_reviews.replace(',', '')
    num_reviews = int(num_reviews)  # convert text into integer
    print('[parse] Number of Reviews:', num_reviews)

    url_template = url.replace('.html', '-or{}.html')
    print('[parse] URL Template:', url_template)

    items = []

    offset = 0

    while (True):
        subpage_url = url_template.format(offset)

        subpage_items = parse_reviews(session, subpage_url)
        if not subpage_items:
            break

        items += subpage_items

        if len(subpage_items) < 10:
            break

        offset += 10 # This changes how far ahead you skip for reviews

    return items

In [18]:
def get_reviews_ids(soup):
    items = soup.find_all('div', attrs={'data-reviewid': True})

    if items:
        reviews_ids = [x.attrs['data-reviewid'] for x in items][::2]
        print('[get_reviews_ids] data-reviewid:', reviews_ids)
        return reviews_ids

In [19]:
def get_more(session, reviews_ids):
    #
    # Initial Source: Furas, https://stackoverflow.com/questions/47856273/scraping-reviews-from-tripadvisor/47858268#47858268
    #
    
    url = 'https://www.tripadvisor.com/OverlayWidgetAjax?Mode=EXPANDED_HOTEL_REVIEWS_RESP&metaReferer=Hotel_Review'

    payload = {
        'reviews': ','.join(reviews_ids),
        'widgetChoice': 'EXPANDED_HOTEL_REVIEW_HSX',
        'haveJses': 'earlyRequireDefine,amdearly,global_error,long_lived_global,apg-Hotel_Review,apg-Hotel_Review-in,bootstrap,desktop-rooms-guests-dust-en_US,responsive-calendar-templates-dust-en_US,taevents',
        'haveCsses': 'apg-Hotel_Review-in',
        'Action': 'install',
    }

    soup = post_soup(session, url, payload)

    return soup

In [20]:
def parse_reviews(session, url):
    '''Get all reviews from one page'''

    print('Parsing Reviews -- url:', url)

    soup = get_soup(session, url)

    if not soup:
        # Error
        print('Parsing Reviews -- no soup:', url)
        return

    venue_name = soup.find('h1', id='HEADING').text

    reviews_ids = get_reviews_ids(soup)
    if not reviews_ids:
        return

    soup = get_more(session, reviews_ids)

    if not soup:
        # Error
        print('Parsing Reviews -- no soup:', url)
        return

    items = []

    for idx, review in enumerate(soup.find_all('div', class_='reviewSelector')):

        if review:
        
            # Contributions
            badgets = review.find_all('span', class_='badgetext')
            if len(badgets) > 0:
                contributions = badgets[0].text
            else:
                contributions = '0'

            #print('contributions: ', contributions)

            # Helpful Votes
            if len(badgets) > 1:
                helpful_vote = badgets[1].text
            else:
                helpful_vote = '0'

            #print('helpful: ', helpful_vote)


            # User Location
            user_loc = review.select_one('div.userLoc strong')
            if user_loc:
                user_loc = user_loc.text
            else:
                user_loc = ''
            #print('user loc: ', user_loc)

            # User Name
            soup.find_all(href=re.compile("elsie"))

            if review.find(id = re.compile("UID_")):
                uid = review.find(id = re.compile("UID_"))['id']
                uid = re.split(r'[_-]', uid)[1]
            else:
                uid = ''


            # Review Rating
            if review.select_one('span.ui_bubble_rating'):
                bubble_rating = review.select_one('span.ui_bubble_rating')['class']
                bubble_rating = bubble_rating[1].split('_')[-1]
            else:
                bubble_rating = ''

            # Review ID
            if review:
                review_id = review.attrs['data-reviewid']
                review_title = review.select_one('span.noQuotes').string
            else:
                review_id = ''
                review_title = ''


            item = {
                'venue_name': venue_name,
                'review_id': review_id,
                'review_title': review_title,
                'review_body': review.find('p', class_='partial_entry').text,
                'review_date': review.find('span', class_='ratingDate')['title'], # 'ratingDate' instead of 'relativeDate'
                'rating': bubble_rating,
                'helpful_vote': helpful_vote,
                'contributions': contributions,
                'uid': uid,
                'user_location': user_loc
            }

            items.append(item)

            # Slow down process a bit
            #time.sleep(.5)
            
        else:
            break

    return items

## Write out Reviews & Related Metadata

In [21]:
def write_reviews(items, filename='results.dat',
                 headers=['review_body'],
                 mode='w'):
    print('--- Review Write Start ---')

    with io.open(filename, mode, encoding="utf-8") as csvfile:
        csv_file = csv.DictWriter(csvfile, headers, delimiter='\t', extrasaction='ignore')

        csv_file.writerows(items)
        
    print('--- Review Write Finish ---')
    
def write_metas(items, filename='results_meta.dat',
                headers=['venue_name', 'review_id', 'review_title',
                'review_date', 'rating', 'helpful_vote', 'contributions',
                'uid', 'user_location'], 
                mode='w'):

    print('--- Meta Write Start ---')
    
    with io.open(filename, mode, encoding="utf-8") as csvfile:
        
        meta_file = csv.DictWriter(csvfile, headers, delimiter='\t', extrasaction='ignore')
        meta_file.writerows(items)
    
    print('--- Meta Write Finish ---')
    
def write_both(items, filename='results_both.dat',
                headers=['venue_name', 'review_id', 'review_title',
                'review_date', 'rating', 'helpful_vote', 'contributions',
                'uid', 'user_location', 'review_body'], 
                mode='w'):

    print('--- Both Write Start ---')
    
    with io.open(filename, mode, encoding="utf-8") as csvfile:
        
        meta_file = csv.DictWriter(csvfile, headers, delimiter='\t', extrasaction='ignore')
        meta_file.writerows(items)
    
    print('--- Both Write Finish ---')


## Actual Scraping

Results of scraping will be output here for tracking purposes and the datasets are stored in their respective directories

In [22]:
for url in start_urls:

    # get all reviews for 'url' and 'lang'
    %time items = scrape(url, lang)


    if not items:
        print('No reviews')
    else:
        # write in CSV
        filename = dataset_name_stem
        print('Dataset Name:', filename)
        write_reviews(items, './' + filename + '/' + filename + '.dat', mode='w')
        write_metas(items, './' + filename + '/' + 'metadata.dat', mode='w')
        write_both(items, './' + filename + '/' + 'both.dat', mode = 'w')

[parse] Number of Reviews: 36551
[parse] URL Template: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or{}.html?filterLang=en
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or0.html?filterLang=en
[get_reviews_ids] data-reviewid: ['657934522', '657921383', '657899498', '657874889', '657851781', '657845814', '657807479', '657793838', '657685946', '657653977']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or10.html?filterLang=en
[get_reviews_ids] data-reviewid: ['657650091', '657512715', '657470411', '657457755', '657441834', '657410802', '657406266', '657396538', '657394138', '657373397']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or20.html?filterLang=en
[get_r

[get_reviews_ids] data-reviewid: ['651078128', '651075654', '651050482', '650969481', '650922685', '650795218', '650739995', '650665022', '650559222', '650512795']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or260.html?filterLang=en
[get_reviews_ids] data-reviewid: ['650454896', '650434344', '650414263', '650406009', '650386999', '650370567', '650317925', '650262049', '650210658', '650204526']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or270.html?filterLang=en
[get_reviews_ids] data-reviewid: ['650200578', '650195392', '650194125', '650187709', '650163775', '650150292', '650143208', '650125205', '650120896', '650065010']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or280.html?filterLang=en
[get_reviews_ids] data-revie

Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or510.html?filterLang=en
[get_reviews_ids] data-reviewid: ['644306467', '644276349', '644268089', '644180589', '644179003', '644170352', '644145644', '644130170', '644123464', '644120844']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or520.html?filterLang=en
[get_reviews_ids] data-reviewid: ['644118625', '644113134', '644104427', '644085305', '644079844', '644076633', '644071730', '644061741', '644035624', '644035066']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or530.html?filterLang=en
[get_reviews_ids] data-reviewid: ['644004723', '643892280', '643874652', '643816581', '643804119', '643792650', '643671778', '643640643', '643635122', '643631250']
Parsing Reviews -- url: http

[get_reviews_ids] data-reviewid: ['638080852', '638071802', '638034266', '638018573', '637883578', '637821259', '637818373', '637735636', '637733731', '637722428']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or770.html?filterLang=en
[get_reviews_ids] data-reviewid: ['637628512', '637531983', '637529497', '637507899', '637483642', '637342870', '637338675', '637284218', '637250577', '637107490']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or780.html?filterLang=en
[get_reviews_ids] data-reviewid: ['637032060', '636983828', '636939371', '636935900', '636918152', '636883068', '636867632', '636747053', '636670231', '636536541']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or790.html?filterLang=en
[get_reviews_ids] data-revie

Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or1020.html?filterLang=en
[get_reviews_ids] data-reviewid: ['630436497', '630410486', '630390891', '630387594', '630385612', '630334009', '630315467', '630256331', '630253591', '630251555']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or1030.html?filterLang=en
[get_reviews_ids] data-reviewid: ['630244151', '630226228', '630225455', '630196835', '630194954', '630185833', '630181389', '630179839', '630177144', '630164009']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or1040.html?filterLang=en
[get_reviews_ids] data-reviewid: ['630157843', '630118662', '630107065', '630093801', '630081906', '630077799', '630073089', '630042561', '630027342', '629965553']
Parsing Reviews -- url: h

...

[get_reviews_ids] data-reviewid: ['119404604', '119354269', '119334860', '119288465', '119231246', '119162634', '119148523', '119146962', '119120819', '119115508']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or26590.html?filterLang=en
[get_reviews_ids] data-reviewid: ['119112913', '119082882', '119053782', '119016188', '119014005', '119012771', '119002404', '118994859', '118907446', '118841750']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or26600.html?filterLang=en
[get_reviews_ids] data-reviewid: ['118832929', '118817120', '118816043', '118760238', '118728877', '118719279', '118708468', '118696996', '118680207', '118628525']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or26610.html?filterLang=en
[get_reviews_ids] data

Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or26840.html?filterLang=en
[get_reviews_ids] data-reviewid: ['108620381', '108397232', '108332689', '108199307', '107728821', '107680045', '107504801', '107496214', '107281584', '107242032']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or26850.html?filterLang=en
[get_reviews_ids] data-reviewid: ['106758485', '106752250', '106629605', '106588344', '106419639', '106249561', '106172422', '106132594', '106131187', '106036706']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or26860.html?filterLang=en
[get_reviews_ids] data-reviewid: ['105952237', '105857020', '105674431', '105611553', '105428204', '105409044', '105406264', '105302602', '105296716', '105283714']
Parsing Reviews -- url

Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or27100.html?filterLang=en
[get_reviews_ids] data-reviewid: ['82523405', '82517058', '82468753', '82467474', '82436001', '82383294', '82288853', '82269914', '82180619', '81989054']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or27110.html?filterLang=en
[get_reviews_ids] data-reviewid: ['81940719', '81908076', '81662616', '81369095', '81241367', '81218567', '81161110', '81134702', '80533056', '80305379']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or27120.html?filterLang=en
[get_reviews_ids] data-reviewid: ['80304398', '80221621', '80183677', '80129682', '80011863', '79522376', '79446377', '79269777', '79170787', '79146456']
Parsing Reviews -- url: https://www.tripadvisor.com/

Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or27360.html?filterLang=en
[get_reviews_ids] data-reviewid: ['46357953', '46326485', '46324208', '46320058', '46311588', '46309147', '46273722', '46263908', '46262698', '46237241']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or27370.html?filterLang=en
[get_reviews_ids] data-reviewid: ['46214675', '46191819', '46137941', '45979829', '45811111', '45647370', '45480056', '45376327', '44361020', '44093191']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or27380.html?filterLang=en
[get_reviews_ids] data-reviewid: ['43307492', '41657248', '41402190', '41314167', '40605787', '39600455', '39344006', '39011492', '38970297', '38474103']
Parsing Reviews -- url: https://www.tripadvisor.com/

[get_reviews_ids] data-reviewid: ['7476060', '7430965', '7429376', '7428434', '7419508', '7415214', '7391541', '7389010', '7302616', '7299657']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or27630.html?filterLang=en
[get_reviews_ids] data-reviewid: ['7294132', '7290019', '7284159', '7252719', '7215954', '7209527', '6933275', '6918087', '6821687', '6765714']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or27640.html?filterLang=en
[get_reviews_ids] data-reviewid: ['6720543', '6687012', '6654733', '6611323', '6598955', '6582603', '6568278', '6552916', '6542167', '6515614']
Parsing Reviews -- url: https://www.tripadvisor.com/Attraction_Review-g34515-d102432-Reviews-Universal_Studios_Florida-Orlando_Florida-or27650.html?filterLang=en
[get_reviews_ids] data-reviewid: ['6495476', '6480948', '6438304', '6363318', '615

*** 

In [23]:
write_both(items, './' + filename + '/' + 'both.dat', mode = 'w')

--- Both Write Start ---
--- Both Write Finish ---
